# 2 - Nettoyage et pre-processing de la BDD

In [ ]:
import pandas as pd

# Chargement de la base de données contenant les infos + performances des joueurs

df = pd.read_csv('./data/merged_data.csv')

print(f"Dataset chargé: {len(df)} lignes, {len(df.columns)} colonnes")

Dataset chargé: 10284 lignes, 78 colonnes


Grâce au scraping mis en place, on récupère un jeu de données assez conséquant (10k+ données et presque 80 colonnes). Afin de réduire le volume de données à traîter pour accélérer le temps de calcul des modèles sans perdre trop d'information, on peut commencer par supprimer les données très corrélées (>= à 80% en valeur absolue). Commençons d'abord par calculer le nombre de paires de variables qui sont largement corrélées :

In [2]:
from clean_dataset_fun import analyze_correlations

# Analyse des corrélations élevées (>= 80%)

correlations_list = analyze_correlations(df, threshold=0.80, verbose=True)

Variables avec une corrélation >= 80% (en valeur absolue):

Nombre total de paires trouvées: 172

  1. NBA_FANTASY_PTS_RANK <-> WNBA_FANTASY_PTS_RANK : +0.9957
  2. WNBA_FANTASY_PTS     <-> NBA_FANTASY_PTS      : +0.9949
  3. FGM_RANK             <-> PTS_RANK             : +0.9924
  4. FGM                  <-> PTS                  : +0.9895
  5. FG3A                 <-> FG3M                 : +0.9857
  6. MIN_RANK             <-> MIN                  : -0.9835
  7. FGA_RANK             <-> PTS_RANK             : +0.9834
  8. FGA                  <-> PTS                  : +0.9821
  9. FTA_RANK             <-> FTM_RANK             : +0.9821
 10. FTM                  <-> FTA                  : +0.9821
 11. W_RANK               <-> W                    : -0.9816
 12. FGM                  <-> FGA                  : +0.9793
 13. FGM_RANK             <-> FGA_RANK             : +0.9780
 14. adjusted_salary      <-> Salary               : +0.9779
 15. FG3A_RANK            <-> FG3M_RANK        

On remarque qu'il y a 172 paires de variables qui sont extrêmement corrélées. On va donc supprimer une partie de ces variables qui portent vraissemblablement la même information que les autres (tout en "protégeant certaines colonnes pour qu'elles ne soient pas supprimées, comme celles du salaire car elles seront les targets de nos modèles).

In [3]:
from clean_dataset_fun import remove_highly_correlated_features

# Suppression des colonnes hautement corrélées, en protégeant certaines colonnes qu'on souhaite conserver

protected_columns = ['Salary', 'adjusted_salary', 'next_adjusted_salary']

columns_to_drop = remove_highly_correlated_features(df, threshold=0.80, protected_cols=protected_columns, verbose=False)

df_cleaned = df.drop(columns=columns_to_drop)

print(f"Dataset chargé: {len(df)} lignes, {len(df.columns)} colonnes")

print(f"Dataset nettoyé: {len(df_cleaned)} lignes, {len(df_cleaned.columns)} colonnes")

Dataset chargé: 10284 lignes, 78 colonnes
Dataset nettoyé: 10284 lignes, 42 colonnes


En procédant de cette manière, on arrive à diviser le nombre de variables par 2, ce qui va considéramment améliorer la performance. Maintenant, concentrons nous sur la suppressions des outliers de cette nouvelle base de données. En effet, il y a certaines données qui vont compromettre la qualité de nos modèles car elles sont associées à des cas très particuliers. Par exemple, on va supprimer les informations sur les joueurs dont le salaire ajusté à l'inflation est inférieur 500k$, car c'était le salaire minimum en NBA en 2000. Les joueurs gagnant un montant inférieur ne jouent pas en NBA de manière régulière ou à plein temps, on supprime donc les points associés. De plus, il arrive qu'un joueur voit son salaire drastiquement diminuer car il arrive en fin de carrière, or nous on s'intéresse plutôt à l'augmentation ou baisse des salaires des joueur actifs, en fonction de leur qualité de jeu. On supprime donc les points associés à une baisse considérable de salaire qui indique généralement une fin de carrière (ainsi que les points suivants car un joueur peut jouer quelques saisons avec un salaire de fin de carrière).

In [4]:
from clean_dataset_fun import clean_outliers

df_clean = clean_outliers(df_cleaned, min_salary=500000, drop_threshold=-0.70)

NETTOYAGE DES OUTLIERS

Dataset initial: 10284 lignes

1. Suppression des salaires ajustés < 500,000$
   Lignes supprimées: 404
   Lignes restantes: 9880

2. Détection des fins de carrière (baisse > 70%)
   Joueurs avec baisse > 70%: 343
   Lignes identifiées (fin de carrière): 804
   Lignes supprimées (fin de carrière): 804
   Lignes restantes: 9076

RÉSUMÉ DU NETTOYAGE
Dataset initial:                10284 lignes
Supprimés (salaire < 500,000$):    404 lignes
Supprimés (fin de carrière):    804 lignes
Total supprimé:                 1208 lignes (11.75%)
Dataset final:                  9076 lignes

 Nettoyage des outliers terminé


De cette manière, on supprime plus de 1000 données qui vont contribuer à dégrader la qualité de nos modèles. Maintenant que nous avons supprimé les variables et points de mauvaise qualité, on va appliqué un pre-processing du dataset en plusieurs étapes : 

1. Suppression des colonnes inutiles pour les modèles ('PLAYER_ID', 'PLAYER_NAME'...)
2. Encodage des variables catégorielles (comme la position des joueurs, ID de l'équipe)
3. Normalisation des variables numériques
4. Vérification des valeurs manquantes (on supprime les données qui n'ont pas de 'next_adjusted_salary', car c'est la variable target de nos modèles. Sinon on remplace par la valeur moyenne)

In [5]:
from clean_dataset_fun import preprocess_pipeline

df_preprocessed, artifacts = preprocess_pipeline(df_clean, verbose=True)

print(f"Dataset preprocessed: {len(df_preprocessed)} lignes, {len(df_preprocessed.columns)} colonnes")

PIPELINE DE PRÉTRAITEMENT DES DONNÉES

1. Dataset initial: (9076, 42)

2. Colonnes supprimées: ['PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID', 'Team', 'Salary', 'Season', 'Year', 'adjusted_salary', 'Rank']
Nouvelles dimensions: (9076, 32)

3. Encodage des variables catégorielles:

Position encodée:
   Nombre de classes: 9
   Classes: ['C', 'F', 'G', 'GF', 'PF', 'PG', 'SF', 'SG', 'nan']
   Mapping: {'C': np.int64(0), 'F': np.int64(1), 'G': np.int64(2), 'GF': np.int64(3), 'PF': np.int64(4), 'PG': np.int64(5), 'SF': np.int64(6), 'SG': np.int64(7), 'nan': np.int64(8)}

TEAM_ABBREVIATION encodée:
   Nombre de classes: 36

Colonnes catégorielles originales supprimées: ['Position', 'TEAM_ABBREVIATION']

4. Colonnes à normaliser (31):
   ['AGE', 'W', 'L', 'FG_PCT', 'FG3M', 'FG3_PCT', 'FTM', 'OREB', 'DREB', 'AST']...

Normalisation effectuée avec StandardScaler
   Moyenne ≈ 0, Écart-type ≈ 1

5. Valeurs manquantes détectées:
   next_adjusted_salary: 1427 (15.72%)
   Changed_team: 1427 (15.7

Maintenant que les données sont prêtes à être utilisées, on crée un jeu d'entraînement et un jeu de test

In [9]:
# Création d'un jeu d'entraînement et d'un jeu de test

from sklearn.model_selection import train_test_split

X = df_preprocessed.drop(columns=['next_adjusted_salary'])

y = df_preprocessed['next_adjusted_salary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"Jeu d'entraînement: {len(X_train)} lignes, {len(X_train.columns)} colonnes")

print(f"Jeu de test: {len(X_test)} lignes, {len(X_test.columns)} colonnes")

Jeu d'entraînement: 5354 lignes, 31 colonnes
Jeu de test: 2295 lignes, 31 colonnes
